<h1><center>Laboratorio 9: Optimización de modelos 💯</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Primavera 2024</strong></center>

### **Cuerpo Docente:**

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Luis Picón
- Nombre de alumno 2: Israel Astudillo Martínez


### **Link de repositorio de GitHub:** [Insertar Repositorio](https://github.com/IsraPKMNPAP/Laboratorio-de-Herramientas)

### Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.

### Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Optimizar modelos usando `optuna`
- Recurrir a técnicas de *prunning*
- Forzar el aprendizaje de relaciones entre variables mediante *constraints*
- Fijar un pipeline con un modelo base que luego se irá optimizando.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

# Importamos librerias útiles

In [ ]:
!pip install -qq xgboost optuna

# El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Al ver el gran potencial y talento que usted ha demostrado en el campo de la ciencia de datos, Fiu lo contrata como data scientist para que forme parte de su nuevo emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv("sales.csv")

df.head()

,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


## 1 Generando un Baseline (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad. [0.5 puntos]
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas. [1 punto]
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas. `Nota:` Utilice el método `.set_output(transform='pandas')` para obtener un DataFrame como salida del `ColumnTransformer` [1 punto]
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios. [0.5 punto]
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio? [0.5 puntos]
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`? [1 punto]
7. Guarde ambos modelos en un archivo .pkl (uno cada uno) [0.5 puntos]

In [ ]:
from sklearn import set_config
set_config(transform_output="pandas")

# Inserte su código acá

In [ ]:
# Paquetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error

In [ ]:
# Separación en train 0.7, validation 0.2 y test 0.1
X = df.drop(columns=["quantity"])
y = df["quantity"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
# Function Transformer

# Función que añade las componentes de la fecha al dataframe
def extract_date_features(df):
    df["date"] = pd.to_datetime(df["date"], format="%d/%m/%y")
    df["day"] = df["date"].dt.day.astype("category")
    df["month"] = df["date"].dt.month.astype("category")
    df["year"] = df["date"].dt.year.astype("category")
    return df.drop(columns=["date"])


In [ ]:
# Instanciación del transformer
date_transformer = FunctionTransformer(extract_date_features)

In [ ]:
# Columnas numéricas y categóricas
num = ["price"]
date = ["date"]
cat = ["city","brand","container","capacity","lat","long"]

In [ ]:
# Pipelines

# Numérico
num_pipe= Pipeline([("pass","passthrough")])

# Categórico
cat_pipe = Pipeline([
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

# Fechas
date_pipe = Pipeline([
    ("date", date_transformer),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

# ColumnTransformer
preprocessor = ColumnTransformer([
    ("date", date_pipe, date),
    ("num", num_pipe, num),
    ("cat", cat_pipe, cat)
])

In [ ]:
# Aplicamos el preprocesador
preprocessor.fit_transform(X_train)

,date__day_28,date__day_29,date__day_30,date__day_31,date__month_1,date__month_2,date__month_3,date__month_4,date__month_5,date__month_6,...,cat__lat_37.97945,cat__lat_38.24444,cat__lat_39.63689,cat__lat_40.64361,cat__long_21.73444,cat__long_22.41761,cat__long_22.93086,cat__long_23.68708,cat__long_23.71622,cat__long_25.14341
4164,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
5470,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1527,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6319,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6455,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4715,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2051,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4566,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
7312,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [ ]:
preprocessor.set_output(transform='pandas')

ColumnTransformer(transformers=[('date',
                                 Pipeline(steps=[('date',
                                                  FunctionTransformer(func=<function extract_date_features at 0x7b9e682f5a20>)),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 ['date']),
                                ('num',
                                 Pipeline(steps=[('pass', 'passthrough')]),
                                 ['price']),
                                ('cat',
                                 Pipeline(steps=[('onehot',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 ['city', 'brand', 'container', 'capacity',
                                  'lat', 'long'])])

In [ ]:
# Pipeline completo
model_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("regressor", DummyRegressor(strategy="mean"))  # Promedio como estrategia
])

In [ ]:
# Salida en dataframe
model_pipeline.set_output(transform='pandas')

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('date',
                                                  Pipeline(steps=[('date',
                                                                   FunctionTransformer(func=<function extract_date_features at 0x7b9e682f5a20>)),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['date']),
                                                 ('num',
                                                  Pipeline(steps=[('pass',
                                                                   'passthrough')]),
                                                  ['price']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['city', 'brand', 'container',
                                                   'capacity', 'lat',
                                                   'long'])])),
                ('regressor', DummyRegressor())])

In [ ]:
# Entrenar el pipeline
model_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('date',
                                                  Pipeline(steps=[('date',
                                                                   FunctionTransformer(func=<function extract_date_features at 0x7b9e682f5a20>)),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['date']),
                                                 ('num',
                                                  Pipeline(steps=[('pass',
                                                                   'passthrough')]),
                                                  ['price']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['city', 'brand', 'container',
                                                   'capacity', 'lat',
                                                   'long'])])),
                ('regressor', DummyRegressor())])

In [ ]:
# Predicciones y cálculo de MAE
y_val_pred = model_pipeline.predict(X_val)
mae = mean_absolute_error(y_val, y_val_pred)
print(f"MAE (DummyRegressor): {mae}")

MAE (DummyRegressor): 13576.56699051175


Esta métrica se interpreta como la diferencia absoluta entre promedio entre el valor predicho y el real. Es decir, en promedio el modelo se equivoca en 13576 unidades tanto por arriba como por abajo del valor real, lo cual es bastante alto.

In [ ]:
# Importamos el regresor
from xgboost import XGBRegressor
# Instanciamos XGBoost
xgb_model = XGBRegressor()
# Pipeline con regresor
xgb_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("regressor xgb", xgb_model)
])

In [ ]:
# Salida en dataframe
xgb_pipeline.set_output(transform='pandas')

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('date',
                                                  Pipeline(steps=[('date',
                                                                   FunctionTransformer(func=<function extract_date_features at 0x7b9e682f5a20>)),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['date']),
                                                 ('num',
                                                  Pipeline(steps=[('pass',
                                                                   'passthrough')]),
                                                  ['price']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHo...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=None, n_jobs=None,
                              num_parallel_tree=None, random_state=None, ...))])

In [ ]:
# Entrenar el pipeline
xgb_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('date',
                                                  Pipeline(steps=[('date',
                                                                   FunctionTransformer(func=<function extract_date_features at 0x7b9e682f5a20>)),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['date']),
                                                 ('num',
                                                  Pipeline(steps=[('pass',
                                                                   'passthrough')]),
                                                  ['price']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHo...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=None, n_jobs=None,
                              num_parallel_tree=None, random_state=None, ...))])

In [ ]:
# Predicciones y cálculo de MAE
y_val_pred_xgb = xgb_pipeline.predict(X_val)
mae_xgb = mean_absolute_error(y_val, y_val_pred_xgb)
print(f"MAE (XGBRegressor): {mae_xgb}")

MAE (XGBRegressor): 2450.481110847121


Vemos que el MAE disminuye drásticamente y el desempeño es mucho mejor que utilizando las medias como predictor.

In [ ]:
import pickle
# Guardamos el pickle de ambos modelos
with open('dummy_pipeline_model.pkl', 'wb') as f:
    pickle.dump(model_pipeline, f)
# Guardar el modelo entrenado
with open('xgb_pipeline_model.pkl', 'wb') as f:
    pickle.dump(xgb_pipeline, f)

In [ ]:
import xgboost as xgb
print(xgb.__version__)

2.1.1


## 2. Forzando relaciones entre parámetros con XGBoost (10 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo realizando las siguientes tareas:

1. Vuelva a entrenar el `Pipeline` con `XGBRegressor`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Para aplicar esta restricción apóyese en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>. [6 puntos]

>Hint 1: Para implementar el constraint se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

>Hint 2: Puede obtener el nombre de las columnas en el paso anterior al modelo regresor mediante el método `.get_feature_names_out()`

2. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. [1 puntos]

3. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo? [2 puntos]

4. Guarde su modelo en un archivo .pkl [1 punto]

In [ ]:
# Obtener los nombres de las características
fitted = preprocessor.fit_transform(X_train)
feature_names = fitted.columns.tolist()

# Crear la lista de restricciones monotónicas basada en los nombres de características
constraints = {}
for feature in feature_names:
    if 'price' in feature:  # Monotonicidad negativa para las características relacionadas con 'price'
        constraints[feature] = -1
    else:
        constraints[feature] = 0  # Sin restricciones para otras características

# Actualizamos el pipeline con las restricciones monotónicas
xgb_pipeline_monotonic = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor xgb', XGBRegressor(monotone_constraints=constraints, enable_categorical=False))
])

# Entrenar el modelo con las restricciones
xgb_pipeline_monotonic.fit(X_train, y_train)

# Predicciones en el conjunto de validación
y_pred_xgb_monotonic = xgb_pipeline_monotonic.predict(X_val)

# Calcular MAE con las restricciones monotónicas
mae_xgb_monotonic = mean_absolute_error(y_val, y_pred_xgb_monotonic)
print('MAE con restricción monotónica: ' + str(mae_xgb_monotonic))

MAE con restricción monotónica: 2597.1949489269455


In [ ]:
# Comparamos el MAE para el modelo con restricción y sin restricción
print(f"MAE sin restricciones: {mae_xgb}")
print(f"MAE con restricciones: {mae_xgb_monotonic}")


MAE sin restricciones: 2450.481110847121
MAE con restricciones: 2597.1949489269455


Vemos que el MAE sube ligeramente con esta restricción. Podemos decir que la relación sugerida no ayudó a mejorar el desempeño del modelo y lo empeoró ligeramente.

In [ ]:
# Guardamos el modelo con restricciones monotónicas
with open('xgb_monotonic_model.pkl', 'wb') as f:
    pickle.dump(xgb_pipeline_monotonic, f)


## 1.3 Optimización de Hiperparámetros con Optuna (20 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se pide que su optimización considere lo siguiente:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)

Para ello se pide los siguientes pasos:
1. Implemente una función `objective()` que permita minimizar el `MAE` en el conjunto de validación. Use el método `.set_user_attr()` para almacenar el mejor pipeline entrenado. [10 puntos]
2. Fije el tiempo de entrenamiento a 5 minutos. [1 punto]
3. Optimizar el modelo y reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
4. Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados? [5 puntos]
5. Guardar su modelo en un archivo .pkl [1 punto]

In [ ]:
import optuna
from optuna.samplers import TPESampler
optuna.logging.set_verbosity(optuna.logging.WARNING)
import joblib
import time

# Fijar la semilla para reproducibilidad
seed = 42


def objective(trial):
    # Inserte su código acá
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1)
    n_estimators = trial.suggest_int('n_estimators', 50, 1000)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    max_leaves = trial.suggest_int('max_leaves', 0, 100)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 5)
    reg_alpha = trial.suggest_float('reg_alpha', 0.0, 1.0)
    reg_lambda = trial.suggest_float('reg_lambda', 0.0, 1.0)
    #min_frequency = trial.suggest_float('min_frequency', 0.0, 1.0)

    # Actualizar OneHotEncoder con min_frequency optimizado
    #preprocessor.set_params(cat__min_frequency=min_frequency)

    # Definir el pipeline con los hiperparámetros sugeridos
    pipeline_xgb_optuna = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', XGBRegressor(
            learning_rate=learning_rate,
            n_estimators=n_estimators,
            max_depth=max_depth,
            max_leaves=max_leaves,
            min_child_weight=min_child_weight,
            reg_alpha=reg_alpha,
            reg_lambda=reg_lambda,
            enable_categorical=True,
            random_state=seed
        ))
    ])

    # Entrenar el modelo
    pipeline_xgb_optuna.fit(X_train, y_train)

    # Predecir en el conjunto de validación
    y_pred = pipeline_xgb_optuna.predict(X_val)

    # Calcular MAE
    mae = mean_absolute_error(y_val, y_pred)

    # Almacenar el mejor pipeline en Optuna
    trial.set_user_attr("best_pipeline", pipeline_xgb_optuna)

    return mae

# Configuración de Optuna y ejecución de la optimización
study = optuna.create_study(sampler=optuna.samplers.TPESampler(), direction='minimize')
study.optimize(objective, n_trials=100, timeout=300)  # 5 minutos de límite de tiempo

# Resultados
best_trial = study.best_trial
print(f"Number of trials: {len(study.trials)}")
print(f"Best MAE: {best_trial.value}")
print(f"Best hyperparameters: {best_trial.params}")

# Guardar el mejor modelo en un archivo .pkl
import joblib
best_pipeline = best_trial.user_attrs["best_pipeline"]
joblib.dump(best_pipeline, "best_xgb_model_optuna.pkl")

Number of trials: 100
Best MAE: 1938.3844627232559
Best hyperparameters: {'learning_rate': 0.07567143188358535, 'n_estimators': 897, 'max_depth': 8, 'max_leaves': 87, 'min_child_weight': 4, 'reg_alpha': 0.7843685937504801, 'reg_lambda': 0.32835297534559726}


['best_xgb_model_pruning.pkl']

Notamos que al utilizar el modelo con la optimización de hiperparámetros el MAE disminuye considerablemente respecto a los dos modelos anteriores, esto debido obviamente a que la optimización de estos hiperparámetros ayuda a reducir el MAE al equilibrar la capacidad del modelo de aprender patrones complejos sin sobreajustar a los datos de entrenamiento

**4**

*learning_rate*: Controla la tasa de aprendizaje del modelo. Un valor más bajo puede requerir más iteraciones pero reduce el riesgo de overfitting.

*n_estimators*: Es el número de árboles a entrenar. Tener más árboles puede mejorar la precisión, pero también incrementaría el costo computacional.

*max_depth*: Corresponde a la profundidad máxima de cada árbol.Sirve para evitar el overfitting si es muy alto.

*max_leaves*: Número máximo de hojas permitidas en un árbol. Controla la complejidad del modelo.

*min_child_weight*: Es el peso mínimo de las instancias necesarias en una hoja del árbol. Valores más altos de este hiperparámetro hacen que el modelo sea más conservador.

*reg_alpha*: Término de regularización L1, puede ayudar a reducir el overfitting.

*reg_lambda*: Término de regularización L2, también ayuda a reducir el overfitting.

*min_frequency*: En OneHotEncoder, controla la frecuencia mínima de las categorías antes de ser agrupadas. Optimizar este valor puede mejorar el rendimiento en variables categóricas.

## 4. Optimización de Hiperparámetros con Optuna y Prunners (17 puntos)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento? [2 puntos]
- Redefinir la función `objective()` utilizando `optuna.integration.XGBoostPruningCallback` como método de **Prunning** [10 puntos]
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos [1 punto]
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
- Guardar su modelo en un archivo .pkl [1 punto]

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

El prunning es un método que ayuda a disminuir la complejidad de los modelos basados en árboles de decisión, eliminando ramas innecesarias que puedan generar overfitting. Esto puede significar menor complejidad y cómputo para el modelo y mayor capacidad de generalización, dependiendo de la técnica usada ya se pre prunning limitando el largo de las ramas o post prunning eliminado las ramas menos relevantes luego del entrenamiento.

In [33]:
!pip install optuna-integration[xgboost]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 2.4 MB/s eta 0:00:00


In [41]:
# Inserte su código acá
from optuna.integration import XGBoostPruningCallback
# Fijar la semilla para reproducibilidad
seed = 42

def objective(trial):
    # Sugerir hiperparámetros
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1)
    n_estimators = trial.suggest_int('n_estimators', 50, 1000)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    max_leaves = trial.suggest_int('max_leaves', 0, 100)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 5)
    reg_alpha = trial.suggest_float('reg_alpha', 0.0, 1.0)
    reg_lambda = trial.suggest_float('reg_lambda', 0.0, 1.0)
    verbosity = 0

    # Definir el pipeline con los hiperparámetros sugeridos
    pipeline_xgb_pruning = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', XGBRegressor(
            learning_rate=learning_rate,
            n_estimators=n_estimators,
            max_depth=max_depth,
            max_leaves=max_leaves,
            min_child_weight=min_child_weight,
            reg_alpha=reg_alpha,
            reg_lambda=reg_lambda,
            random_state=seed
        ))
    ])

    # Entrenamiento del modelo con callback de prunning
    pipeline_xgb_pruning.fit(X_train, y_train,
                             regressor__eval_set=[(X_val, y_val)],

                             regressor__callbacks=[XGBoostPruningCallback(trial, "validation_0-mae")])

    # Predecir en el conjunto de validación
    y_pred = pipeline_xgb_pruning.predict(X_val)

    # Calcular MAE
    mae = mean_absolute_error(y_val, y_pred)

    # Almacenar el mejor pipeline en Optuna
    trial.set_user_attr("best_pipeline", pipeline_xgb_pruning)

    return mae

# Silenciar los prints obtenidos durante pruning
optuna.logging.set_verbosity(optuna.logging.WARNING)
# Configuración de Optuna y ejecución de la optimización
study = optuna.create_study(sampler=TPESampler(), direction='minimize')
study.optimize(objective, n_trials=100, timeout=300, show_progress_bar=True)  # 5 minutos de límite de tiempo

# Resultados
best_trial = study.best_trial
print(f"Number of trials: {len(study.trials)}")
print(f"Best MAE: {best_trial.value}")
print(f"Best hyperparameters: {best_trial.params}")

# Guardar el mejor modelo en un archivo .pkl
best_pipeline = best_trial.user_attrs["best_pipeline"]
joblib.dump(best_pipeline, "best_xgb_model_pruned.pkl")


  0%|          | 0/100 [00:00<?, ?it/s]

[W 2024-10-25 02:58:03,662] Trial 0 failed with parameters: {'learning_rate': 0.07835048714839916, 'n_estimators': 491, 'max_depth': 5, 'max_leaves': 13, 'min_child_weight': 3, 'reg_alpha': 0.46527576720896024, 'reg_lambda': 0.4741178601071142} because of the following error: ValueError("Invalid parameter 'regressor' for estimator Pipeline(steps=[('preprocessor',\n                 ColumnTransformer(transformers=[('date',\n                                                  Pipeline(steps=[('date',\n                                                                   FunctionTransformer(func=<function extract_date_features at 0x7b9e682f5a20>)),\n                                                                  ('onehot',\n                                                                   OneHotEncoder(handle_unknown='ignore',\n                                                                                 sparse_output=False))]),\n                                                  ['date'])

ValueError: Invalid parameter 'regressor' for estimator Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('date',
                                                  Pipeline(steps=[('date',
                                                                   FunctionTransformer(func=<function extract_date_features at 0x7b9e682f5a20>)),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['date']),
                                                 ('num',
                                                  Pipeline(steps=[('pass',
                                                                   'passthrough')]),
                                                  ['price']),
   

## 5. Visualizaciones (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

1. Gráfico de historial de optimización [1 punto]
2. Gráfico de coordenadas paralelas [1 punto]
3. Gráfico de importancia de hiperparámetros [1 punto]

Comente sus resultados:

4. ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? [0.5 puntos]
5. ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? [1 punto]
6. ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo? [0.5 puntos]

In [42]:
# Inserte su código acá
import optuna.visualization as vis

# Gráfico de historial de optimización
opt_history = vis.plot_optimization_history(study)
opt_history.show()


# Gráfico de coordenadas paralelas
parallel_coords = vis.plot_parallel_coordinate(study)
parallel_coords.show()

# Gráfico de importancia de hiperparámetros
hyper_importance = vis.plot_param_importances(study)
hyper_importance.show()


[W 2024-10-25 03:00:03,627] There are no complete trials.


[W 2024-10-25 03:00:04,014] Your study does not have any completed trials.


[W 2024-10-25 03:00:04,027] Study instance does not contain completed trials.


## 6. Síntesis de resultados (3 puntos)

Finalmente:

1. Genere una tabla resumen del MAE en el conjunto de validación obtenido en los 5 modelos entrenados desde Baseline hasta XGBoost con Constraints, Optuna y Prunning. [1 punto]
2. Compare los resultados de la tabla y responda, ¿qué modelo obtiene el mejor rendimiento? [0.5 puntos]
3. Cargue el mejor modelo, prediga sobre el conjunto de **test** y reporte su MAE. [0.5 puntos]
4. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto? [1 punto]

In [ ]:
# Inserte su código acá

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/8CT1AXElF_cAAAAC/gojo-satoru.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>